In [3]:
%pip install optimum-intel

     ---------------------------------------- 0.0/96.0 kB ? eta -:--:--
     ------------ --------------------------- 30.7/96.0 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 96.0/96.0 kB 922.5 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/258.1 kB ? eta -:--:--
   ----------------- ---------------------- 112.6/258.1 kB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 258.1/258.1 kB 2.7 MB/s eta 0:00:00
 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import torch.utils.benchmark as benchmark
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from optimum.intel import OVModelForSequenceClassification
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score

c:\Users\MgDuck\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

#Либо это....
#model = torch.load('baseline.pt')


#.....либо это
model = AutoModelForSequenceClassification.from_pretrained('mgduck/bert_temp_experiment')


model.to('cpu')

# Убеждаемся, что модель в режиме оценки
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [15]:
#torch.save(model.state_dict(), "baseline.bin")

In [3]:
model_optim = OVModelForSequenceClassification.from_pretrained('mgduck/bert_temp_experiment',  export=True)

Framework not specified. Using pt to export to ONNX.
Using framework PyTorch: 2.0.1+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Compiling the model...
Set CACHE_DIR to C:\Users\MgDuck\AppData\Local\Temp\tmprod83qdt\model_cache


In [4]:

dataset = load_dataset("mteb/tweet_sentiment_extraction")
dataset.set_format(type="torch", columns=["text", "label"])
test_dataset = dataset['test']
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
true = []
preds = []
with torch.no_grad():
    tq = tqdm(test_loader)
    for batch_num, batch in enumerate(tq):


        tokenized_batch = tokenizer(batch["text"], return_tensors="pt", truncation=True, padding=True, max_length=50)
        ids = tokenized_batch['input_ids'].to('cpu')
        mask = tokenized_batch['attention_mask'].to('cpu')
        y = batch['label']


        input = {'input_ids': ids, 'attention_mask': mask}
        logits = model(**input).logits
        y_pred = torch.argmax(logits, axis=1).cpu()

        preds.extend(list(y_pred))
        true.extend(list(y.numpy()))
        tq.set_description(
            f'\rF1 for batch {batch_num}: {round(f1_score(y, y_pred, average="weighted"), 3)}')
f1 = f1_score(true, preds, average="weighted")
print(f'\nTotal F1 score: {round(f1, 2)}')

F1 for batch 110: 0.857: 100%|██████████| 111/111 [01:21<00:00,  1.37it/s]


Total F1 score: 0.77


In [6]:
dataset = load_dataset("mteb/tweet_sentiment_extraction")
dataset.set_format(type="torch", columns=["text", "label"])
test_dataset = dataset['test']
test_loader = DataLoader(test_dataset, batch_size=32)

In [7]:
true = []
preds = []
with torch.no_grad():
    tq = tqdm(test_loader)
    for batch_num, batch in enumerate(tq):


        y = batch['label']

        input = tokenizer(batch["text"], return_tensors="pt", truncation=True, padding=True, max_length=50)
        logits = model_optim(**input).logits
        y_pred = torch.argmax(logits, axis=1).cpu()

        preds.extend(list(y_pred))
        true.extend(list(y.numpy()))
        tq.set_description(
            f'\rF1 for batch {batch_num}: {round(f1_score(y, y_pred, average="weighted"), 3)}')
f1 = f1_score(true, preds, average="weighted")
print(f'\nTotal F1 score: {round(f1, 2)}')

F1 for batch 110: 0.857: 100%|██████████| 111/111 [00:50<00:00,  2.18it/s]


Total F1 score: 0.77
